In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
os.listdir()

['.ipynb_checkpoints',
 'MotionSenseHAR.txt',
 'MotionSenseHAR_TEST.ts',
 'MotionSenseHAR_TRAIN.ts',
 'Untitled.ipynb',
 '실습_MotionSense_DataPreprocessing.ipynb',
 '실습_MotionSense_Modeling.ipynb']

# 데이터 전처리
Sample1 X1 [t1, t2, t3....] : X2 [t1, t2, t3....] ....<br>
Sample2 X1 [t1, t2, t3....] : X2 [t1, t2, t3....] .... <br>

이런 데이터를 (Sample, Time, Feature)의 numpy array로 만들기

In [3]:
Train = pd.read_csv("MotionSenseHAR_TRAIN.ts", skiprows=10, sep=":")

Train_y = Train.iloc[:, 12]
Train_y = pd.get_dummies(Train_y)

Train_x = []
for RowIndex in range(Train.shape[0]):
    Features = []
    for ColIndex in range(0, 12):
        Features.append([float(Digit) for Digit in Train.iloc[RowIndex, ColIndex].split(", ")])
    Train_x.append(np.stack(Features).transpose())
Train_x = np.stack(Train_x)
Train_x.shape

(965, 1000, 12)

In [4]:
Test = pd.read_csv("MotionSenseHAR_Test.ts", skiprows=10, sep=":")

Test_y = Test.iloc[:, 12]
Test_y = pd.get_dummies(Test_y)

Test_x = []
for RowIndex in range(Test.shape[0]):
    Features = []
    for ColIndex in range(0, 12):
        Features.append([float(Digit) for Digit in Test.iloc[RowIndex, ColIndex].split(", ")])
    Test_x.append(np.stack(Features).transpose())
Test_x = np.stack(Test_x)
Test_x.shape

(264, 1000, 12)

### Data Shuffle

In [5]:
arr = np.arange(Train_x.shape[0])
np.random.shuffle(arr)

Train_x = Train_x[arr]
Train_y = Train_y.iloc[arr]

In [6]:
arr = np.arange(Test_x.shape[0])
np.random.shuffle(arr)

Test_x = Test_x[arr]
Test_y = Test_y.iloc[arr]

### 1D convoltuion 딥러닝으로 Time series 데이터 분석

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, SimpleRNN, TimeDistributed, LSTM, GRU, Bidirectional
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau

In [8]:
TimeDomain = 1000
nFeatures = 12
n_feature_maps = 64

In [10]:
input_layer = tf.keras.layers.Input( (TimeDomain, nFeatures, ) )
conv_x = tf.keras.layers.Conv1D(filters=n_feature_maps, kernel_size=8, padding='same')(input_layer)
conv_x = tf.keras.layers.BatchNormalization()(conv_x)
conv_x = tf.keras.layers.Activation('relu')(conv_x)
conv_x = tf.keras.layers.Dropout(rate=0.1)(conv_x)

conv_y = tf.keras.layers.Conv1D(filters=n_feature_maps, kernel_size=5, padding='same')(conv_x)
conv_y = tf.keras.layers.BatchNormalization()(conv_y)
conv_y = tf.keras.layers.Activation('relu')(conv_y)
conv_x = tf.keras.layers.Dropout(rate=0.1)(conv_x)

conv_z = tf.keras.layers.Conv1D(filters=n_feature_maps, kernel_size=3, padding='same')(conv_y)
conv_z = tf.keras.layers.BatchNormalization()(conv_z)


# expand channels for the sum
shortcut_y = tf.keras.layers.Conv1D(filters=n_feature_maps, kernel_size=1, padding='same')(input_layer)
shortcut_y = tf.keras.layers.BatchNormalization()(shortcut_y)

output_block_1 = tf.keras.layers.add([shortcut_y, conv_z])
output_block_1 = tf.keras.layers.Activation('relu')(output_block_1)
conv_x = tf.keras.layers.Dropout(rate=0.1)(conv_x)

# BLOCK 2

conv_x = tf.keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_1)
conv_x = tf.keras.layers.BatchNormalization()(conv_x)
conv_x = tf.keras.layers.Activation('relu')(conv_x)
conv_x = tf.keras.layers.Dropout(rate=0.1)(conv_x)

conv_y = tf.keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
conv_y = tf.keras.layers.BatchNormalization()(conv_y)
conv_y = tf.keras.layers.Activation('relu')(conv_y)
conv_x = tf.keras.layers.Dropout(rate=0.1)(conv_x)

conv_z = tf.keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
conv_z = tf.keras.layers.BatchNormalization()(conv_z)


# expand channels for the sum
shortcut_y = tf.keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=1, padding='same')(output_block_1)
shortcut_y = tf.keras.layers.BatchNormalization()(shortcut_y)

output_block_2 = tf.keras.layers.add([shortcut_y, conv_z])
output_block_2 = tf.keras.layers.Activation('relu')(output_block_2)
conv_x = tf.keras.layers.Dropout(rate=0.1)(conv_x)

# BLOCK 3

conv_x = tf.keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_2)
conv_x = tf.keras.layers.BatchNormalization()(conv_x)
conv_x = tf.keras.layers.Activation('relu')(conv_x)
conv_x = tf.keras.layers.Dropout(rate=0.1)(conv_x)

conv_y = tf.keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
conv_y = tf.keras.layers.BatchNormalization()(conv_y)
conv_y = tf.keras.layers.Activation('relu')(conv_y)
conv_x = tf.keras.layers.Dropout(rate=0.1)(conv_x)

conv_z = tf.keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
conv_z = tf.keras.layers.BatchNormalization()(conv_z)

# no need to expand channels because they are equal
shortcut_y = tf.keras.layers.BatchNormalization()(output_block_2)

output_block_3 = tf.keras.layers.add([shortcut_y, conv_z])
output_block_3 = tf.keras.layers.Activation('relu')(output_block_3)

# FINAL

gap_layer = tf.keras.layers.GlobalAveragePooling1D()(output_block_3)
output_layer = tf.keras.layers.Dense(6, activation='softmax')(gap_layer)

model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)

In [11]:
model.compile(loss = "categorical_crossentropy", optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 1000, 12)]   0           []                               
                                                                                                  
 conv1d_11 (Conv1D)             (None, 1000, 64)     6208        ['input_2[0][0]']                
                                                                                                  
 batch_normalization_12 (BatchN  (None, 1000, 64)    256         ['conv1d_11[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_9 (Activation)      (None, 1000, 64)     0           ['batch_normalization_12[0]

In [65]:
FinalModel.compile(loss = "categorical_crossentropy", optimizer=Adam(learning_rate=0.005), metrics=['accuracy'])

In [ ]:
history_finetunning = model.fit(
    Train_x, Train_y,
    epochs=100,
    batch_size=8,
    #callbacks=CALLBACK,
    validation_data=(Test_x, Test_y),
    shuffle=True)

Epoch 1/100
121/121 [==============================] - 28s 210ms/step - loss: 0.5850 - accuracy: 0.7948 - val_loss: 1.0962 - val_accuracy: 0.7008
Epoch 2/100
121/121 [==============================] - 26s 216ms/step - loss: 0.3098 - accuracy: 0.9088 - val_loss: 1.5423 - val_accuracy: 0.7500
Epoch 3/100
  6/121 [>.............................] - ETA: 22s - loss: 0.1904 - accuracy: 0.9792